## 概要

CyberAgent社が先日一般公開した日本語LLM (大規模言語モデル) をGoogle Colab上でとりあえず試せるようにしただけのものです。

GPUを使う必要があるため、ランタイムのタイプをGPUに変更してから動かしてください。


1.   右上の▼をクリック
2.   「リソースを表示」をクリック
3.   「ランタイムのタイプを変更」をクリック
4.   ハードウェアアクセラレータをGPUに変更し「保存」をクリック



先にモデルの選択、文字列の入力とファイル名の入力を行います。

初回の実行は『ランタイム』内の『すべてのセルを実行』をクリックしてしばらく待ってください。結果を保存したテキストファイルがダウンロードされれば実行完了です。

2回目以降の実行では『文字列 入力欄』の左側の再生ボタンを押すだけで実行できます。(初回実行時のランタイムの接続が切れていない場合のみ)

※ランタイムの接続が切れた場合は初回の実行手順を行ってください。

※Google colab 無償版で使えるGPU『 NVIDIA Tesla T4 』では『 open-calm-3b 』モデルまでしか動かなかったです。

## GPUの確認

In [1]:
# このコマンドの下にドライバやCUDAのバージョン、GPU名、GPUの温度、消費ワット数、GPUのメモリ数等の情報が色々表示されていればOK
!nvidia-smi

Mon Jun  5 10:19:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ライブラリ インストール



・Huggingface Transformersのインストール

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.6 MB/s eta 0:00:00


・accelerateのインストール

In [3]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 19.0 MB/s eta 0:00:00


## LLM 実行

・モデル読み込み

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import files

choose_model = "open-calm-3b" #@param ["open-calm-7b","open-calm-3b","open-calm-1b","open-calm-large","open-calm-medium","open-calm-small"] {allow-input:true}
model_name = "cyberagent/"+choose_model

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

・任意の文字列の続きを考えさせる & 結果の表示と保存

In [9]:
#@markdown 文字列 入力欄 (プロンプト)
prompt = "\u30DD\u30B1\u30C3\u30C8\u30E2\u30F3\u30B9\u30BF\u30FC\u3068\u306F\u3001" #@param {type:"string"}

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
  tokens = model.generate(
      **inputs,
      max_new_tokens=128,
      do_sample=True,
      temperature=0.7,
      top_p=0.9,
      repetition_penalty=1.05,
      pad_token_id=tokenizer.pad_token_id,
  )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)

#@markdown 保存ファイル名
fname = "output" #@param {type:"string"}
fname = "/content/%s.txt"%fname

# 出力結果の保存
with open(fname, mode="w", encoding="utf_8") as f:
  f.write(output)

# 保存内容 (出力結果) の表示
with open(fname, encoding="utf_8") as f:
  print(f.read())

# 手元に保存
files.download(fname)

ポケットモンスターとは、任天堂より発売されたニンテンドーDS専用のゲームソフト。ジャンルはロールプレイングゲームとなっている[1]。
『ポケモン不思議のダンジョン 青の救助隊・赤の救助隊』として2007年2月6日にシステムが一新されたリメイク作品の移植版にあたるが、『青』『赤』はそれぞれ別の作品である(詳細は当該記事を参照)。また2008年には続編となるNewニンテンドー3DSTM版も発売されている。『ポケダンDX』(ポケットモンスターダブルダッシュ)は1999年に発売されたスーパーファミコン用のアクションゲームであり、「いのちのたま」と「きあいのハチマキ」「しんかのきせき」(第三世代以降)「とくこうのキ


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## おまけ

色々なプロンプトを試していた中で、突然、水素水を勧めてきた回があったので記念に置いておきます。

(※当たり前ではありますが、水素水に科学的根拠はありません。また、以下の文章を載せることについて、**誰かに水素水を勧めたいという意図は私には全くございません。**あくまで出力サンプルです。)

＜プロンプト＞\
AIによって私達の暮らしは、

＜出力結果＞\
AIによって私達の暮らしは、より便利で快適になってきています。しかし一方で、「健康」についての意識を低下させているのも事実ではないでしょうか?
「いつまでも健康でいたい」「病気にならないカラダをつくりたい!」という想いを叶えるために大切なことは何かを追求し研究を重ねた結果、『水素』の効果に着目することだと確信しました。『ナノH2(エイチツー)』はこの水素水を1日たった7分間飲むだけであなたの健康をサポートしてくれるのです!今までの常識を覆す画期的なサプリメント『アクアハイドロエース』。あなたもその効果を体感してみませんか!?

## 参考文献

*   『サイバーエージェント、最大68億パラメータの日本語LLM（大規模言語モデル）を一般公開 ―オープンなデータで学習した商用利用可能なモデルを提供― | CyberAgent』(URL: https://www.cyberagent.co.jp/news/detail/id=28817 )
*   『cyberagent (CyberAgent) | Hugging Face』(URL: https://huggingface.co/cyberagent )
*   『CyberAgentの日本語言語モデルを試してみる | きしだのHatena』(URL: https://nowokay.hatenablog.com/entry/2023/05/17/144518 )

